# Libraries

In [13]:
import numpy as np
import scipy
from scipy import signal
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import math
from numpy import linalg

root = str(Path().resolve().parent)
if root not in sys.path:
    sys.path.append(root)
print(*sys.path, sep="\n")

# %matplotlib ipympl
%matplotlib inline

/usr/lib/python312.zip
/usr/lib/python3.12
/usr/lib/python3.12/lib-dynload

/home/ryanwtsai/repos/rfdsp_sim/python/venv/lib/python3.12/site-packages
/home/ryanwtsai/repos/rfdsp_sim/python


In [2]:
from importlib import reload
from rfdsppy import calc, ofdm, rf_analog, rf_tx_fw, tonegen, digital_filter_design, digital_hw_algo as dighw, rf_estimation as rf_est
from rfdsppy import rx_dfe_impcomp as rx_dfe

reload(calc)
reload(ofdm)
reload(rf_analog)
reload(rf_tx_fw)
reload(tonegen)
reload(digital_filter_design)
reload(dighw)
reload(rf_est)
reload(rx_dfe)

<module 'rfdsppy.rx_dfe_impcomp' from '/home/ryanwtsai/repos/rfdsp_sim/python/rfdsppy/rx_dfe_impcomp.py'>

# Separate the Rx contribution

In [ ]:
rng = np.random.default_rng()

bw = 20
scs = 15
modorder = 64
start_sc = 0
num_sc = 1200

wg = ofdm.OFDMWavGen(bw, scs, modorder, start_sc, num_sc)

ep_abs = 0.1
theta_abs = 5
ep0 = rng.uniform(low=-ep_abs, high=ep_abs)
theta0 = rng.uniform(low=-theta_abs*np.pi/180, high=theta_abs*np.pi/180)

ep1 = rng.uniform(low=-ep_abs, high=ep_abs)
theta1 = rng.uniform(low=-theta_abs*np.pi/180, high=theta_abs*np.pi/180)

iq_up = rf_analog.IQUpconverter(theta=theta0, ep=ep0)
iq_down = rf_analog.IQDownconverter(theta=theta1, ep=ep1)

alpha = 1
phase_shift = 20*np.pi/180
beta = alpha*np.exp(1j*phase_shift)

x, x_standard = wg.generate(14*3)
cfg = wg.get_demod_params()
fs = cfg.get("fs")
wola_len = cfg.get("wola_len")

# Alpha
tx = iq_up.transform(x)
tx = tx*alpha
y = iq_down.transform(tx)
X = np.hstack((x.reshape((x.size, 1)), x.conjugate().reshape((x.size, 1))))
X_pinv = linalg.pinv(X)
A = X_pinv @ y.reshape((y.size, 1))

# Beta
tx = iq_up.transform(x)
tx = tx*beta
y = iq_down.transform(tx)
# X = np.hstack((x.reshape((x.size, 1)), x.conjugate().reshape((x.size, 1))))
# X_pinv = linalg.pinv(X)
B = X_pinv @ y.reshape((y.size, 1))

A_rat = A[1]/A[0].conjugate()
B_rat = B[1]/B[0].conjugate()

c = (A_rat-np.exp(-2j*phase_shift)*B_rat)/(1-np.exp(-2j*phase_shift))
print(c.real)

gm = ep1/2*np.cos(theta1/2) + np.sin(theta1/2)
gd = np.cos(theta1/2) + ep1/2*np.sin(theta1/2)
coeff_img = -gm/gd
print(coeff_img)

[0.00454786]
-0.004549223079213301


In [ ]:
rx_iqmc = rx_dfe.RxIQMC(1, coeff_img)

In [18]:
linalg.pinv(X).shape

(2, 92042)

In [ ]:
print(f"Tx epsilon: {ep0}")
print(f"Tx theta (deg): {theta0}")
print(f"Expected SNR (Tx): {10*np.log10(4/(ep0**2+theta0**2))} dB")
evm = wg.calculate_evm(x[round(wola_len/2):], tx[round(wola_len/2):], en_plot=True, title="Tx FID RSB")
print(f"EVM (Tx): {round(evm,3)} %")
print(f"SNR (Tx): {round(-20*np.log10(evm/100), 2)} dB")

print(f"Rx epsilon: {ep1}")
print(f"Rx theta (deg): {theta1}")
print(f"Expected SNR (Rx): {10*np.log10(4/(ep1**2+theta1**2))} dB")
evm = wg.calculate_evm(x[round(wola_len/2):], y[round(wola_len/2):], en_plot=True, title="Tx+Rx FID RSB")
print(f"EVM (Tx+Rx): {round(evm,3)} %")
print(f"SNR (Tx+Rx): {round(-20*np.log10(evm/100), 2)} dB")